# Normalize data to MNI template

This example covers the normalization of data. Some people prefer to normalize the data during the preprocessing, just before smoothing. I prefer to do the 1st-level analysis completely in subject space and only normalize the contrasts for the 2nd-level analysis. But both approaches are fine.

For the current example, we will take the computed 1st-level contrasts from the previous experiment (again once done with fwhm=4mm and fwhm=8mm) and normalize them into MNI-space. To show two different approaches, we will do the normalization once with ANTs and once with SPM.

## Normalization with ANTs

The normalization with ANTs requires that you first compute the transformation matrix that would bring the anatomical images of each subject into template space. Depending on your system this might take a few hours per subject. To facilitate this step, the transformation matrix is already computed for the T1 images.

## Imports (ANTs)

First, we need to import all the modules we later want to use.

In [1]:
from os.path import join as opj
from nipype import Workflow, Node, MapNode
from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.interfaces.fsl import Info

## Experiment parameters (ANTs)

It's always a good idea to specify all parameters that might change between experiments at the beginning of your script. And remember that we decided to run the group analysis without subject ``sub-01``, ``sub-06`` and ``sub-10`` because they are left-handed (see [this section](https://miykael.github.io/nipype_tutorial/notebooks/example_1stlevel.html#Special-case)).

In [2]:
experiment_dir = '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized'
output_dir = 'datasink'

# list of subject identifiers (remember we use only right handed subjects)
subject_list = ['%02d'%i for i in range(1, 23)]

# Template to normalize to
template = '/home/neuro/Data/Visual_search_task/mni_icbm152_nlin_asym_09c/1mm_T1.nii.gz'

## Specify Nodes (ANTs)

Initiate all the different interfaces (represented as nodes) that you want to use in your workflow.

In [ ]:
# Apply Transformation - applies the normalization matrix to contrast images
apply2con = MapNode(ApplyTransforms(args='--float',
                                    input_image_type=3,
                                    interpolation='BSpline',
                                    invert_transform_flags=[False],
                                    num_threads=1,
                                    reference_image=template,
                                    terminal_output='file'),
                    name='apply2con', iterfield=['input_image'])

## Specify input & output stream (ANTs)

Specify where the input data can be found & where and how to save the output data.

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']), name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles - to grab the data (alternativ to DataGrabber)
input_data_dir = '/home/neuro/Result/Visual_search_task/'
templates = {'con': opj('1st_analysis/analysis_result/datasink/1stLevel/', 'sub-{subject_id}', 'con_00??.nii'),
             'transform': opj('preproc_working_dir/warp_field/antsdir/transform/', '{subject_id}', 'transformComposite.h5')}
selectfiles = Node(SelectFiles(templates, base_directory=input_data_dir, sort_filelist=True), name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir, container=output_dir), name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-')]
subjFolders = [('_apply2con%s/' % (i), '') for i in range(23)] # number of contrast used in 1stlevel an.
substitutions.extend(subjFolders)
datasink.inputs.substitutions = substitutions

## Specify Workflow (ANTs)

Create a workflow and connect the interface nodes and the I/O stream to each other.

In [ ]:
# Initiation of the ANTs normalization workflow
antsflow = Workflow(name='antsflow', base_dir=experiment_dir)

# Connect up the ANTs normalization components
antsflow.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (selectfiles, apply2con, [('con', 'input_image'),
                                            ('transform', 'transforms')]),
                  (apply2con, datasink, [('output_image', 'norm_ants.@con')]),
                  ])

## Visualize the workflow (ANTs)

It always helps to visualize your workflow.

In [ ]:
# Create ANTs normalization graph
antsflow.write_graph(graph2use='flat', format='png', simple_form=True)

##  Run the Workflow (ANTs)

Now that everything is ready, we can run the ANTs normalization workflow. Change ``n_procs`` to the number of jobs/cores you want to use.

In [ ]:
antsflow.run('MultiProc', plugin_args={'n_procs': 4})

# Normalization with SPM12

The normalization with SPM12 is rather straightforward. The only thing we need to do is run the Normalize12 module. **So let's start!**

## Imports (SPM12)

First, we need to import all the modules we later want to use.

In [2]:
from os.path import join as opj
from nipype.interfaces.spm import Normalize12
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype import Workflow, Node

## Experiment parameters (SPM12)

It's always a good idea to specify all parameters that might change between experiments at the beginning of your script. And remember that we decided to run the group analysis without subject ``sub-01``, ``sub-06`` and ``sub-10`` because they are left-handed (see [this section](https://miykael.github.io/nipype_tutorial/notebooks/example_1stlevel.html#Special-case)).

In [3]:
experiment_dir = '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized'
output_dir = 'datasink_spm'

# list of subject identifiers
subject_list = ['%02d'%i for i in range(1, 23)]
subject_list.remove('11')

template = '/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii'

## Specify Nodes (SPM12)

Initiate all the different interfaces (represented as nodes) that you want to use in your workflow.

In [4]:
# Normalize - normalizes functional and structural images to the MNI template
normalize = Node(Normalize12(jobtype='estwrite',
                             tpm=template,
                             write_voxel_sizes=[3.7, 3.7, 3.7]), name="normalize")

stty: 'standard input': Inappropriate ioctl for device


## Specify input & output stream (SPM12)

Specify where the input data can be found & where and how to save the output data.

In [5]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']), name="infosource")
infosource.iterables = [('subject_id', subject_list)]

templates = {'anat': '/home/neuro/Data/Visual_search_task/BIDS_data/sub-{subject_id}/ses-test/anat/sub-{subject_id}_ses-test_anat_sT1W.nii',
             'func': '/home/neuro/Data/Visual_search_task/BIDS_data/sub-{subject_id}/ses-test/func/sub-{subject_id}_ses-test_session*_bold.nii'}

input_data_dir = '/home/neuro/Result/Visual_search_task/'
selectfiles = Node(SelectFiles(templates, base_directory=input_data_dir, sort_filelist=True),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir, container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', 'sub-')]
datasink.inputs.substitutions = substitutions

## Specify Workflow (SPM12)

Create a workflow and connect the interface nodes and the I/O stream to each other.

In [7]:
# Specify Normalization-Workflow & Connect Nodes
spmflow = Workflow(name='spmflow')
spmflow.base_dir = opj(experiment_dir)

# Connect up SPM normalization components
spmflow.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                 (selectfiles, normalize, [('func', 'apply_to_files')]),
                 (selectfiles, normalize, [('anat', 'image_to_align')]),
                 (normalize, datasink, [('normalized_files', 'norm_spm.@files')])
                 ])

## Visualize the workflow (SPM12)

It always helps to visualize your workflow.

In [ ]:
# Create SPM normalization graph
spmflow.write_graph(graph2use='colored', format='png', simple_form=True)

##  Run the Workflow (SPM12)

In [8]:
spmflow.run('MultiProc', plugin_args={'n_procs': 4})

211125-10:29:32,897 nipype.workflow INFO:
	 Workflow spmflow settings: ['check', 'execution', 'logging', 'monitoring']
211125-10:29:34,896 nipype.workflow INFO:
	 Running in parallel.
211125-10:29:34,919 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 21 jobs ready. Free memory (GB): 7.00/7.00, Free processors: 4/4.
211125-10:29:36,408 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.selectfiles" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_22/selectfiles".
211125-10:29:36,882 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.selectfiles" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_21/selectfiles".
211125-10:29:36,901 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.selectfiles" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_20/selectfiles".
211125-10:29:36,969 nipype.workflow INFO:
	 [MultiPr

stty: 'standard input': Inappropriate ioctl for device


211125-10:49:14,88 nipype.workflow INFO:
	 [Node] Finished "spmflow.normalize".
211125-10:49:15,540 nipype.workflow INFO:
	 [Job 7] Completed (spmflow.normalize).
211125-10:49:15,574 nipype.workflow INFO:
	 [MultiProc] Running 3 tasks, and 18 jobs ready. Free memory (GB): 6.40/7.00, Free processors: 1/4.
                     Currently running:
                       * spmflow.normalize
                       * spmflow.normalize
                       * spmflow.normalize
211125-10:49:16,153 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-10:49:16,166 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.datasink" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_20/datasink".
211125-10:49:16,169 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-10:49:16,276 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
211125-10:49:16,279 nipype.i

stty: 'standard input': Inappropriate ioctl for device


211125-10:49:18,610 nipype.workflow INFO:
	 [Node] Finished "spmflow.normalize".
211125-10:49:19,537 nipype.workflow INFO:
	 [Job 10] Completed (spmflow.normalize).
211125-10:49:19,539 nipype.workflow INFO:
	 [Job 12] Completed (spmflow.selectfiles).
211125-10:49:19,541 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 18 jobs ready. Free memory (GB): 6.60/7.00, Free processors: 2/4.
                     Currently running:
                       * spmflow.normalize
                       * spmflow.normalize
211125-10:49:19,589 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-10:49:19,592 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.datasink" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_19/datasink".
211125-10:49:19,594 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-10:49:19,664 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.no

...

211125-11:25:12,505 nipype.workflow INFO:
	 [Node] Finished "spmflow.normalize".
211125-11:25:13,303 nipype.workflow INFO:
	 [Job 55] Completed (spmflow.normalize).
211125-11:25:13,304 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 1 jobs ready. Free memory (GB): 6.60/7.00, Free processors: 2/4.
                     Currently running:
                       * spmflow.normalize
                       * spmflow.normalize
211125-11:25:13,375 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-11:25:13,378 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.datasink" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_03/datasink".
211125-11:25:13,381 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-11:25:13,433 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
211125-11:25:13,434 nipype.interface INFO:
	 sub: /home/neuro/Result/V

stty: 'standard input': Inappropriate ioctl for device


211125-11:26:06,570 nipype.workflow INFO:
	 [Node] Finished "spmflow.normalize".
211125-11:26:07,340 nipype.workflow INFO:
	 [Job 58] Completed (spmflow.normalize).
211125-11:26:07,342 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 6.80/7.00, Free processors: 3/4.
                     Currently running:
                       * spmflow.normalize
211125-11:26:07,392 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-11:26:07,395 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.datasink" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_02/datasink".
211125-11:26:07,398 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-11:26:07,450 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
211125-11:26:07,452 nipype.interface INFO:
	 sub: /home/neuro/Result/Visual_search_task/1st_analysis/analysis_res

stty: 'standard input': Inappropriate ioctl for device


211125-11:26:31,349 nipype.workflow INFO:
	 [Node] Finished "spmflow.normalize".
211125-11:26:31,364 nipype.workflow INFO:
	 [Job 61] Completed (spmflow.normalize).
211125-11:26:31,365 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 7.00/7.00, Free processors: 4/4.
211125-11:26:31,411 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-11:26:31,413 nipype.workflow INFO:
	 [Node] Setting-up "spmflow.datasink" in "/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/spmflow/_subject_id_01/datasink".
211125-11:26:31,416 nipype.workflow INFO:
	 [Node] Outdated cache found for "spmflow.datasink".
211125-11:26:31,484 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
211125-11:26:31,485 nipype.interface INFO:
	 sub: /home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/datasink_spm/norm_spm/_subject_id_01/wsub-01_ses-test_session1_b

# Comparison between ANTs and SPM normalization

Now that we ran the normalization with ANTs and SPM, let us compare their output.

In [ ]:
from nilearn.plotting import plot_stat_map
from nilearn.plotting import plot_glass_brain
anatimg = '/home/neuro/Data/ds000114/derivatives/fmriprep/mni_icbm152_nlin_asym_09c/1mm_T1.nii.gz'

First, let's compare the normalization of the **anatomical** images:

In [ ]:
plot_stat_map(
    '/home/neuro/Result/Visual_search_task/preproc_working_dir/warp_field/antsdir/warped_image/01/transform_Warped.nii.gz',
    title='anatomy - ANTs (normalized to ICBM152)', bg_img=anatimg,
    threshold=200, display_mode='ortho', cut_coords=(-50, 0, -10))
plot_stat_map(
    '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/datasink_spm/norm_spm/sub-01/wsub-01_ses-test_anat_sT1W.nii',
    title='anatomy - SPM (normalized to SPM\'s TPM)', bg_img=anatimg,
    threshold=200, display_mode='ortho', cut_coords=(-50, 0, -10))

And what about the **contrast** images for **Finger > others**?

In [ ]:
plot_stat_map(
    '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/datasink_ants/norm_ants/sub-01/con_0006_trans.nii', title='contrast5 - ANTs',
    bg_img=anatimg, threshold=2, vmax=5, display_mode='ortho', cut_coords=(-39, -37, 56))
plot_stat_map(
    '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/datasink_spm/norm_spm/sub-01/wcon_0006.nii', title='contrast5 - SPM',
    bg_img=anatimg, threshold=2, vmax=5, display_mode='ortho', cut_coords=(-39, -37, 56))

In [ ]:
plot_glass_brain(
    '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/datasink_ants/norm_ants/sub-01/con_0006_trans.nii', colorbar=True,
    threshold=3, display_mode='lyrz', black_bg=True, vmax=6, title='contrast5 - ANTs')
plot_glass_brain(
    '/home/neuro/Result/Visual_search_task/1st_analysis/analysis_result/1st_normalized/datasink_spm/norm_spm/sub-01/wcon_0006.nii', colorbar=True,
    threshold=3, display_mode='lyrz', black_bg=True, vmax=6, title='contrast5 - SPM')